In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [21]:
import pickle

In [22]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [23]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [24]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    # Changing to duration in minutes
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    
    df[categorical] = df[categorical].astype(str)
    
    return df

In [25]:
df_train = read_dataframe("./data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("./data/green_tripdata_2021-02.parquet")

In [26]:
len(df_train), len(df_val)

(73908, 61921)

In [27]:
df_train['PU_DO'] = df_train['PULocationID'] + '-' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '-' + df_val['DOLocationID'] 

In [28]:
# categorical = ['PULocationID', 'DOLocationID']
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [29]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

## Training - Using Linear Regression

In [30]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.479560557870074

In [31]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

## Training - Using Lasso Regression

In [35]:
with mlflow.start_run():

    mlflow.set_tag("developer", "criterian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.001
    mlflow.log_param("alpha", alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    # Saving the model as an artifact
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [ ]:
lr = Ridge(alpha=0.0001)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.34256557347287

## Optimizing parameters using hyper-opt

In [ ]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):

    with mlflow.start_run():
        
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)

        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )

        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[16:29:04] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.86240                          
[1]	validation-rmse:13.72037                          
[2]	validation-rmse:11.47755                          
[3]	validation-rmse:9.91095                           
[4]	validation-rmse:8.83834                           
[5]	validation-rmse:8.11872                           
[6]	validation-rmse:7.63987                           
[7]	validation-rmse:7.31899                           
[8]	validation-rmse:7.10276                           
[9]	validation-rmse:6.95761                           
[10]	validation-rmse:6.85844                          
[11]	validation-rmse:6.78909                          
[12]	validation-rmse:6.73861                          
[13]	validation-rmse:6.70468                          
[14]	validation-rmse:6.67876                          
[15]	validation-rmse:6.65911                          
[

## Using the optimal parameters and Auto logging

In [ ]:
best_params = {
    'learning_rate': 0.3412274574696718,
    'max_depth': 10,
    'min_child_weight': 1.9948769134528168,
    'objective': 'reg:linear',
    'reg_alpha': 0.10987797448608973,
    'reg_lambda': 0.022627747626943775,
    'seed':	42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50
)

2022/06/11 21:28:14 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c6c4ac0e0731424aae7ef9aaa375fa7a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[21:28:14] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:15.23549
[1]	validation-rmse:11.56747
[2]	validation-rmse:9.40998
[3]	validation-rmse:8.18397
[4]	validation-rmse:7.51102
[5]	validation-rmse:7.13908
[6]	validation-rmse:6.93537
[7]	validation-rmse:6.81462
[8]	validation-rmse:6.74215
[9]	validation-rmse:6.69729
[10]	validation-rmse:6.66936
[11]	validation-rmse:6.65355
[12]	validation-rmse:6.64470
[13]	validation-rmse:6.63812
[14]	validation-rmse:6.63105
[15]	validation-rmse:6.62432
[16]	validation-rmse:6.61872
[17]	validation-rmse:6.61441
[18]	validation-rmse:6.61075
[19]	validation-rmse:6.60554
[20]	validation-rmse:6.59986
[21]	validation-rmse:6.59870
[22]	validation-rmse:6.59600
[23]	validation-rmse:6.59309
[24]	validation-rmse:6.59210
[25]	validation-rmse:6.59028
[26]	validation-rmse:6.58861
[27]	validation-rmse:6.58699
[28]	validation-rmse:6.58518
[29]	validation-rmse:6.58382
[30]	validation-rms

2022/06/11 21:29:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/milind/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


## Model Management

Logging models with ml flow

In [38]:
best_params = {
    'learning_rate': 0.3412274574696718,
    'max_depth': 10,
    'min_child_weight': 1.9948769134528168,
    'objective': 'reg:linear',
    'reg_alpha': 0.10987797448608973,
    'reg_lambda': 0.022627747626943775,
    'seed':	42
}

mlflow.log_params(best_params)

booster = xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )

y_pred = booster.predict(valid)
rmse = mean_squared_error(y_val, y_pred, squared=False)
mlflow.log_metric("rmse", rmse)

with open("models/preprocessor.b", "wb") as f_out:
    pickle.dump(dv, f_out)

mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[22:05:31] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:15.23549
[1]	validation-rmse:11.56747
[2]	validation-rmse:9.40998
[3]	validation-rmse:8.18397
[4]	validation-rmse:7.51102
[5]	validation-rmse:7.13908
[6]	validation-rmse:6.93537
[7]	validation-rmse:6.81462
[8]	validation-rmse:6.74215
[9]	validation-rmse:6.69729
[10]	validation-rmse:6.66936
[11]	validation-rmse:6.65355
[12]	validation-rmse:6.64470
[13]	validation-rmse:6.63812
[14]	validation-rmse:6.63105
[15]	validation-rmse:6.62432
[16]	validation-rmse:6.61872
[17]	validation-rmse:6.61441
[18]	validation-rmse:6.61075
[19]	validation-rmse:6.60554
[20]	validation-rmse:6.59986
[21]	validation-rmse:6.59870
[22]	validation-rmse:6.59600
[23]	validation-rmse:6.59309
[24]	validation-rmse:6.59210
[25]	validation-rmse:6.59028
[26]	validation-rmse:6.58861
[27]	validation-rmse:6.58699
[28]	validation-rmse:6.58518
[29]	validation-rmse:6.58382
[30]	validation-rms

2022/06/11 22:06:41 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID f260ba91966e4605a7774aefbe63ea6d. Failed operations: [MlflowException("Changing param values is not allowed. Params were already logged=\'[{\'key\': \'num_boost_round\', \'old_value\': None, \'new_value\': \'1000\'}, {\'key\': \'early_stopping_rounds\', \'old_value\': None, \'new_value\': \'50\'}, {\'key\': \'maximize\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'verbose_eval\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'custom_metric\', \'old_value\': None, \'new_value\': \'None\'}]\' for run ID=\'f260ba91966e4605a7774aefbe63ea6d\'.")]')]


ModelInfo(artifact_path='models_mlflow', flavors={'python_function': {'loader_module': 'mlflow.xgboost', 'python_version': '3.9.12', 'data': 'model.xgb', 'env': 'conda.yaml'}, 'xgboost': {'xgb_version': '1.6.1', 'data': 'model.xgb', 'model_class': 'xgboost.core.Booster', 'code': None}}, model_uri='runs:/f260ba91966e4605a7774aefbe63ea6d/models_mlflow', model_uuid='fcec925e5ec04e46a09d56fd4cd21e72', run_id='f260ba91966e4605a7774aefbe63ea6d', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-06-11 22:06:43.611882', mlflow_version='1.26.1')

In [39]:
logged_model = 'runs:/f260ba91966e4605a7774aefbe63ea6d/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

[22:09:06] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [40]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: f260ba91966e4605a7774aefbe63ea6d

In [41]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

[22:10:26] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [42]:
xgboost_model

In [43]:

y_pred = xgboost_model.predict(valid)

In [44]:
y_pred[:10]

array([14.764206,  7.248774, 13.350018, 24.438328,  9.444439, 17.33367 ,
       11.935371,  8.165582,  8.961153, 19.487   ], dtype=float32)